In [1]:
!pip install sae-lens transformers datasets wandb
!pip install sae-lens transformer_lens transformers datasets torch matplotlib scikit-learn                                                                                        
%pip install sae-lens transformer-lens circuitsvis
!pip install openai
!pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.4/141.4 kB 3.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 15.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 43.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.

In [25]:
from datasets import load_dataset
from transformers import AutoTokenizer
import os
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2Model

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Load SST-2 dataset
dataset = load_dataset("glue", "sst2")

# Load SST-2 train and validation data
def tokenize(batch):
    return tokenizer(batch["sentence"], 
                     padding='max_length',  # Pad all sequences to max_length
                     truncation=True, 
                     max_length=128)

tokenized_train = dataset["train"].map(tokenize, batched=True)
tokenized_val= dataset["validation"].map(tokenize, batched=True)
tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask"])
tokenized_val.set_format(type="torch", columns=["input_ids", "attention_mask"])


# Save tokenized dataset locally
save_dir = "/kaggle/working/tokenized_sst2_gpt2"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

tokenized_train.save_to_disk(f"{save_dir}/train")
tokenized_val.save_to_disk(f"{save_dir}/validation")

print(f"Tokenized SST-2 saved to {save_dir}")


from datasets import load_from_disk
import torch
import os

from sae_lens import (
    LanguageModelSAERunnerConfig,
    SAETrainingRunner,
    StandardTrainingSAEConfig,
    LoggingConfig,
)


import torch

# Device setup
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load local tokenized SST-2 datasets saved with save_to_disk()
train_dataset = load_from_disk("/kaggle/working/tokenized_sst2_gpt2/train")
validation_dataset = load_from_disk("/kaggle/working/tokenized_sst2_gpt2/validation")

# Check that expected columns exist
print("Train dataset columns:", train_dataset.column_names)
print("Validation dataset columns:", validation_dataset.column_names)

# Define training parameters
total_training_steps = 80000
batch_size = 64
total_training_tokens = total_training_steps * batch_size

lr_warm_up_steps = 0
lr_decay_steps = total_training_steps // 5
l1_warm_up_steps = total_training_steps // 20

# Define SAE-Lens config
cfg = LanguageModelSAERunnerConfig(
    model_name="gpt2",  # Correct GPT-2 model ID
    hook_name="blocks.11.hook_mlp_out",  # Last layer MLP output hook
    dataset_path=None,                    # Not used when override_dataset is provided
    is_dataset_tokenized=True,
    streaming=False,
    sae=StandardTrainingSAEConfig(
        d_in=768,
        d_sae=1536,
        apply_b_dec_to_input=False,
        normalize_activations="expected_average_only_in",
        l1_coefficient=0.01,
        l1_warm_up_steps=l1_warm_up_steps,
    ),
    lr=5e-5,
    adam_beta1=0.9,
    adam_beta2=0.999,
    lr_scheduler_name="constant",
    lr_warm_up_steps=lr_warm_up_steps,
    lr_decay_steps=lr_decay_steps,
    train_batch_size_tokens=batch_size,
    context_size=128,
    n_batches_in_buffer=64,
    training_tokens=total_training_tokens,
    store_batch_size_prompts=16,
    feature_sampling_window=1000,
    dead_feature_window=1000,
    dead_feature_threshold=1e-4,
    logger=LoggingConfig(
        log_to_wandb=False,
        wandb_project="sae_lens_sst2_gpt2small_lastlayer_tokenized_local",
        wandb_log_frequency=30,
        eval_every_n_wandb_logs=20,
    ),
    device=device,
    seed=42,
    n_checkpoints=1,
    checkpoint_path="checkpoints",
    dtype="float32",
)

# Instantiate and run SAE training using the loaded dataset
runner = SAETrainingRunner(cfg, override_dataset=train_dataset)
sparse_autoencoder = runner.run()
save_path = "/kaggle/working/saved_sae_model"
os.makedirs(save_path, exist_ok=True)

# If the SAE object has a save method
if hasattr(sparse_autoencoder, "save"):
    sparse_autoencoder.save(save_path)
else:
    # Fallback: save model state dict manually
    torch.save(sparse_autoencoder.state_dict(), os.path.join(save_path, "model_state_dict.pt"))

print(f"SAE model saved to {save_path}")

Saving the dataset (0/1 shards):   0%|          | 0/67349 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/872 [00:00<?, ? examples/s]

Tokenized SST-2 saved to /kaggle/working/tokenized_sst2_gpt2
Train dataset columns: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask']
Validation dataset columns: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask']


/tmp/ipykernel_37/4230650392.py:116: DeprecationWarning: Use LanguageModelSAETrainingRunner instead
  runner = SAETrainingRunner(cfg, override_dataset=train_dataset)


Loaded pretrained model gpt2 into HookedTransformer


/usr/local/lib/python3.11/dist-packages/sae_lens/training/activations_store.py:242: UserWarning: The training dataset contains fewer samples (67349) than the number of samples required by your training configuration (5120000). This will result in multiple training epochs and some samples being used more than once.
  warnings.warn(


Training SAE:   0%|          | 0/5120000 [00:00<?, ?it/s]

Estimating norm scaling factor:   0%|          | 0/1000 [00:00<?, ?it/s]



Refilling buffer:   0%|          | 0/32 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/sae_lens/training/activations_store.py:365: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  yield torch.tensor(


Refilling buffer:   3%|▎         | 1/32 [00:00<00:05,  5.30it/s]

Refilling buffer:   6%|▋         | 2/32 [00:00<00:04,  6.00it/s]

Refilling buffer:   9%|▉         | 3/32 [00:00<00:04,  6.25it/s]

Refilling buffer:  12%|█▎        | 4/32 [00:00<00:04,  6.41it/s]

Refilling buffer:  16%|█▌        | 5/32 [00:00<00:04,  6.47it/s]

Refilling buffer:  19%|█▉        | 6/32 [00:00<00:04,  6.50it/s]

Refilling buffer:  22%|██▏       | 7/32 [00:01<00:03,  6.51it/s]

Refilling buffer:  25%|██▌       | 8/32 [00:01<00:03,  6.54it/s]

Refilling buffer:  28%|██▊       | 9/32 [00:01<00:03,  6.55it/s]

Refilling buffer:  31%|███▏      | 10/32 [00

SAE model saved to /kaggle/working/saved_sae_model


In [26]:
import os
import torch
import numpy as np
from torch.utils.data import DataLoader
from datasets import load_from_disk, load_dataset
from scipy.stats import pearsonr
import random
from kaggle_secrets import UserSecretsClient
from groq import Groq
from tqdm import tqdm
import re
from transformers import GPT2Tokenizer, GPT2Model

# === Initialize Groq Llama 3 client ===
user_secrets = UserSecretsClient()
groq_api_key = user_secrets.get_secret("GROQ_API_KEY")
os.environ["GROQ_API_KEY"] = groq_api_key
client = Groq()

def chat_with_llama(prompt, model="llama-3.1-8b-instant", max_tokens=150):
    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        model=model,
        max_tokens=max_tokens,
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

# --- SAE wrapper class ---
class SAEWithLastLayerActivations(torch.nn.Module):
    def __init__(self, sae_model):
        super().__init__()
        self.sae = sae_model
        self.activations = None
        
        hook_module = getattr(self.sae, "hook_sae_acts_post", None)
        if hook_module is None:
            raise ValueError("hook_sae_acts_post not found in SAE model")
        hook_module.register_forward_hook(self._hook_fn)
        
    def _hook_fn(self, module, input, output):
        self.activations = output.detach()
        
    def forward(self, sae_input):
        _ = self.sae(sae_input)
        return self.activations
    
    def get_activations(self, sae_input):
        self.eval()
        with torch.no_grad():
            _ = self.forward(sae_input)
            return self.activations.cpu()

# --- Setup device and tokenizer ---
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

gpt2_model = GPT2Model.from_pretrained("gpt2").to(device)
gpt2_model.eval()

# --- Function to extract activations with sentences ---
def extract_activations_and_texts(model_wrapper, tokenized_dataset, sentences, feature_idx, device, max_batches=None):
    model_wrapper.eval()
    all_activations = []
    all_texts = []
    all_tokens = []

    dataloader = DataLoader(tokenized_dataset, batch_size=32)
    for i, batch in enumerate(tqdm(dataloader, desc="Extracting activations")):
        if max_batches is not None and i >= max_batches:
            break
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        
        with torch.no_grad():
            outputs = gpt2_model(input_ids=input_ids, attention_mask=attention_mask)
            hidden_states = outputs.last_hidden_state  # (B, seq_len, 768)
            
            activations = model_wrapper.get_activations(hidden_states)  # (B, seq_len, d_sae)

        batch_acts = activations[:, :, feature_idx].cpu().numpy()
        batch_tokens = [tokenizer.convert_ids_to_tokens(ids) for ids in input_ids.cpu().tolist()]
        batch_attn = attention_mask.cpu().numpy()

        start_idx = i * dataloader.batch_size
        batch_texts = sentences[start_idx : start_idx + len(batch["input_ids"])]

        for tokens, acts, attn_mask, text in zip(batch_tokens, batch_acts, batch_attn, batch_texts):
            filtered_tokens = []
            filtered_acts = []
            for t, a, m in zip(tokens, acts, attn_mask):
                if m == 1:
                    filtered_tokens.append(t)
                    filtered_acts.append(a)
            all_tokens.append(filtered_tokens)
            all_activations.append(np.array(filtered_acts))
            all_texts.append(text)

    return all_activations, all_texts, all_tokens

# --- Select top sentences ---
def select_top_sentences(acts, texts, num_top=20):
    scores = [np.max(a) for a in acts]
    idxs = np.argsort(scores)[::-1][:num_top]
    return [acts[i] for i in idxs], [texts[i] for i in idxs]

# --- Generate interpretation ---
def get_llama3_interpretation(top_texts, top_acts):
    prompt = "Given these sentences and their per-token activations, describe in one sentence what causes this feature to activate strongly:\n\n"
    for i, (text, acts) in enumerate(zip(top_texts, top_acts), 1):
        prompt += f"{i}. \"{text}\"\nActivations: {acts.tolist()}\n\n"
    prompt += "Interpretation:"
    print("Sending interpretation prompt to Llama 3...")
    interpretation = chat_with_llama(prompt)
    print("Llama 3 interpretation:", interpretation)
    return interpretation

# --- Select eval sentences ---
def select_eval_sentences(acts, texts, n=5):
    scores = [np.max(a) for a in acts]
    idxs = np.argsort(scores)[::-1]
    high_idxs = idxs[:n]
    remaining = [i for i in range(len(acts)) if i not in high_idxs]
    nonzero = [i for i in remaining if np.std(acts[i]) > 0]
    rand_idxs = random.sample(nonzero, n)
    eval_idxs = list(high_idxs) + rand_idxs
    return [acts[i] for i in eval_idxs], [texts[i] for i in eval_idxs]

# --- Predict activation ---
def get_llama3_activation(sentence, interpretation):
    prompt = (
        f'Feature interpretation: "{interpretation}"\n'
        f'Sentence: "{sentence}"\n'
        'On a scale from 0 to 10, how strongly does this feature activate on the entire sentence?\n'
        'Activation:'
    )
    print(f"Predicting activation for sentence:\n{sentence}")
    response = chat_with_llama(prompt)
    match = re.search(r"\d+(\.\d+)?", response)
    if match:
        return float(match.group())
    else:
        print(f"Warning: Could not parse activation, got response: {response}")
        return 0.0

# --- Compute correlation ---
def compute_correlation(actual, predicted):
    actual_scalar = np.array([np.mean(a) for a in actual])
    predicted_scalar = np.array(predicted)
    corr, _ = pearsonr(actual_scalar, predicted_scalar)
    return corr

# Add this function to save results
def save_top_activations(filename, sentences, actual_acts, predicted_acts):
    with open(filename, "w", encoding="utf-8") as f:
        f.write("Sentence\tActual Activation\tPredicted Activation\n")
        for sent, actual, pred in zip(sentences, actual_acts, predicted_acts):
            actual_score = np.mean(actual) if isinstance(actual, np.ndarray) else actual
            f.write(f"{sent}\t{actual_score:.4f}\t{pred:.4f}\n")
    print(f"Top activations saved to {filename}")

# In your interpretability_score_pipeline, after prediction and before correlation calculation add:
def interpretability_score_pipeline(sparse_autoencoder, validation_tokenized, validation_sentences, feature_index, device):
    print("Extracting per-token activations and sentences...")
    all_acts, all_texts, all_tokens = extract_activations_and_texts(
        sae_with_acts, validation_tokenized, validation_sentences, feature_index, device)

    print("Selecting top sentences for interpretation...")
    top_acts, top_texts = select_top_sentences(all_acts, all_texts, 20)

    print("Getting interpretation from Llama 3...")
    interpretation = get_llama3_interpretation(top_texts[:5], top_acts[:5])

    print("Selecting evaluation sentences for activation prediction...")
    eval_acts, eval_texts = select_eval_sentences(all_acts, all_texts, 5)

    print("Getting predicted activations from Llama 3...")
    predicted_acts = [get_llama3_activation(sent, interpretation) for sent in eval_texts]

    # Save top activating sentences and activations to file
    save_top_activations("top_activations.txt", eval_texts, eval_acts, predicted_acts)

    print("Calculating interpretability score...")
    score = compute_correlation(eval_acts, predicted_acts)
    print(f"\nInterpretability Pearson Correlation Score for feature {feature_index}: {score:.4f}")

# === Main execution ===
if __name__ == "__main__":
    validation_dataset_path = "/kaggle/working/tokenized_sst2_gpt2/validation"
    validation_tokenized = load_from_disk(validation_dataset_path)
    validation_original = load_dataset("glue", "sst2", split="validation")

    # Load your trained SAE model here (replace appropriately)
    # sparse_autoencoder = load_your_sae_model()
    sparse_autoencoder.to(device)

    sae_with_acts = SAEWithLastLayerActivations(sparse_autoencoder)
    sae_with_acts.to(device)
    sae_with_acts.eval()

    interpretability_score_pipeline(
        sparse_autoencoder,
        validation_tokenized,
        validation_original["sentence"],
        feature_index=1311,
        device=device
    )


Extracting per-token activations and sentences...


Extracting activations: 100%|██████████| 28/28 [00:06<00:00,  4.18it/s]


Selecting top sentences for interpretation...
Getting interpretation from Llama 3...
Sending interpretation prompt to Llama 3...
Llama 3 interpretation: The feature that activates strongly in these sentences appears to be related to "film" or "movie" quality, with positive activations indicating praise and negative activations indicating criticism.
Selecting evaluation sentences for activation prediction...
Getting predicted activations from Llama 3...
Predicting activation for sentence:
the terrific and bewilderingly underrated campbell scott gives a star performance that is nothing short of mesmerizing . 
Predicting activation for sentence:
this is a train wreck of an action film -- a stupefying attempt by the filmmakers to force-feed james bond into the mindless xxx mold and throw 40 years of cinematic history down the toilet in favor of bright flashes and loud bangs . 
Predicting activation for sentence:
schaeffer has to find some hook on which to hang his persistently useless movi